<font size="5">Import modules</font>

In [1]:
# Import modules
import pandas as pd
import numpy as np
from nltk.metrics import edit_distance
import time
from tqdm.notebook import tqdm

Object `><:@` not found.


<font size="5">Import data for future clusters name and values from CSV file</font>

In [2]:
#Import data for future clusters name and values from CSV file

clusters = pd.read_csv("sp500names_unique.csv")

<font size="5">Cleaning the Cluster name data</font>

In [3]:
#Cleaning the Cluster name data

cluster_list = clusters['COMNAM'].tolist()
cluster_list2 = [w.replace(' INC', '').replace(' CO', '').replace(' LTD', '') for w in cluster_list]

<font size="5">Import main data from CSV file</font>

<font size="3">File should be uploaded in Jupyter directory</font>

<font size="3">nrows = means how many rows do you need from original file</font>

In [4]:
#Import main data from CSV file

df = pd.read_csv("indivs18.csv", nrows=100, encoding = "ISO-8859-1", low_memory=False)

<font size="5">Insert additional column that will copy and save for further usage Index from original file</font>

In [5]:
#Insert additional column that will copy and save for further usage Index from original file

df.insert(0, column='index1', value=df.index)

<font size="5">Delete all rows where column "Employer" is empty</font>

In [6]:
# Delete all rows where column "Employer" is empty

df = df.dropna(subset=["Employer"])

<font size="5">Make all entries in column "Employer" lowercase</font>

In [7]:
# Make all entries in column "Employer" lowercase

df['Employer'] = df['Employer'].str.lower()

<font size="5">Clean up column "Employer" - select only rows that does NOT contain certain values</font>

In [8]:
# Clean up column "Employer" - select only rows that does NOT contain certain values

df = df[(df.Employer != "not employed") & (df.Employer != "self employed") & (df.Employer != "none") & (df.Employer != "self-employed") & (df.Employer != "self") & (df.Employer != "retired")]

<font size="5">Make column list lowercase</font>

In [ ]:
# Make column list lowercase

column_values1 = list(map(lambda x:x.lower(), cluster_list2))


<font size="5">Add to each column name "_cluster" to use it for Cluster column creation</font>

In [ ]:
#Add to each column name "_cluster" to use it for Cluster column creation

string = '_cluster'
column_names1 = [x + string for x in cluster_list2]


<font size="5">Create a new empty dataframe with Columns for each cluster(company name) and populate columns with values that we will compare later to values in column "Employer"</font>

In [11]:
"""
Create a new empty dataframe with Columns for each cluster(company name) 
and populate columns with values that we will compare later to values in column "Employer"
"""

df1 = pd.DataFrame(columns = column_names1, index=df.index)
i = 0
for col in df1.columns:
    df1[col] = column_values1[i]
    i += 1

<font size="5">Combine two dataframe with clusters with original one</font>

In [13]:
# Combine two dataframe with clusters with original one

df2 = pd.concat([df, df1], axis=1)

<font size="5">Select needed columns for creating Cluster_distance columns</font>

In [ ]:
# Select needed columns for creating Cluster_distance columns

test_list = df2.columns[25:].tolist()
print(test_list[:20])

<font size="5">Changing column names from "cluster" to "distance"</font>

In [ ]:
# Changing column names from "cluster" to "distance"

test_list2 = [w.replace('_cluster', '_distance') for w in test_list]

<font size="5">Create a new empty dataframe with Columns for each cluster_distance</font>

In [16]:
# Create a new empty dataframe with Columns for each cluster_distance

df3 = pd.DataFrame(columns = test_list2, index=df.index)

<font size="5">Populate columns for Cluster_distance with Score metrics. Compare values from each of Cluster columns and value in column "Employer". This line will take 90 % of all time required</font>

In [ ]:
"""
Populate columns for Cluster_distance with Score metrics. 
Compare values from each of Cluster columns and value in column "Employer". This line will take 90 % of all time required
"""

n = 25
for col in tqdm(df3, total=2239, desc="Calculating Score_Metrics"):
    df3[col] = df2.iloc[:, [22,n]].apply(lambda x: edit_distance(*x), axis=1)
    n += 1

<font size="5">Combine all data into a new combined dataframe</font>

In [22]:
#Combine all data into a new combined dataframe

df4 = pd.concat([df2, df3], axis=1)

<font size="5">Selecting data for "Cluster_NAME" final result output column</font>

In [23]:
# Selecting data for "Cluster_NAME" final result output column

df5 = df4.iloc[:, np.r_[0,2,22,25:2263]]

# Selecting data for "Cluster_Score" final result output column

df7 = df4.iloc[:, np.r_[0,2,22,2264:4503]]

<font size="5">Selecting data for "Cluster_Score" final result output column</font>

<font size="5">Grouping data for "Cluster_NAME" final result output column</font>

In [25]:
# Grouping data for "Cluster_NAME" final result output column

df6 = df5.melt(id_vars=['index1', "FECTransID", "Employer"])
group = df6.groupby(["index1", "FECTransID", "Employer", "variable", "value"]).size().reset_index(name='counts')

<font size="5">Grouping data for "Cluster_Score" final result output column</font>

In [26]:
# Grouping data for "Cluster_Score" final result output column

df8 = df7.melt(id_vars=["index1", "FECTransID", "Employer"])
group1 = df8.groupby(["index1", "FECTransID", "Employer", "variable", "value"]).size().reset_index(name='counts')

<font size="5">Adding column for "Cluster_Score" column into combined dataframe</font>

In [27]:
# Adding column for "Cluster_Score" column into combined dataframe

group["Test_value"] = group1["value"]

<font size="5">Drop technically created columns that we don't need in a final result</font>

In [28]:
# Drop technically created columns that we don't need in a final result

group.drop(['variable', "counts"], 1, inplace=True)

<font size="5">Rename columns as we need it</font>

In [29]:
# Rename columns as we need it

group.rename(columns = {'value':'Cluster_NAME', 'Test_value':'Cluster_Score'}, inplace = True)

<font size="5">Select only data that has Score value less than 10</font>

In [30]:
# Select only data that has Score value less than 10

group = group[(group.Cluster_Score <= 10)]

<font size="5">Check before saving if we did everything correct and get a needed output</font>

<font size="5">Save results into file. File will be in Jupyter directory</font>

In [32]:
# Save results into file. File will be in Jupyter directory

group.to_csv("New_structure_100_rows_2239_clusters.csv", index=False)

<font size="5">For speed testing - place needed code lines between """ """. Following lines will test the code for 1M rows from original file and 1 cluster, result displayed in seconds</font>

In [1]:
"""
For speed testing - place needed code lines between """ """. 
Following lines will test the code for 1M rows from original file and 1 cluster, result displayed in seconds

"""

import timeit
s = """
import pandas as pd
import numpy as np
from nltk.metrics import edit_distance
df = pd.read_csv("indivs18.csv", nrows=1000000, low_memory=False, encoding = "ISO-8859-1")
df = df.dropna(subset=["Employer"])
df['Employer'] = df['Employer'].str.lower()
df = df[(df.Employer != "not employed") & (df.Employer != "self employed") & (df.Employer != "none") & (df.Employer != "self-employed") & (df.Employer != "self") & (df.Employer != "retired")]
df["Apple_cluster"] = "apple"
df["ips_ht_distance"] = df.loc[:, ["Employer","Apple_cluster"]].apply(lambda x: edit_distance(*x), axis=1)
"""
timeit.timeit(stmt=s, number=1)

116.24129309999998